# iLuvRags: Retrieval, Rerank, and Evaluation (Colab)

This notebook focuses on retrieval and re-ranking. Ingestion and indexing are assumed to be already done in the repo (`index/` is present).

Sections:
- Setup (install)
- Sanity check (index present)
- Batch queries demo (pretty print + timings)
- Evaluation (Recall@k, MRR) if labels are provided


In [ ]:
# Setup
REPO_URL = "https://github.com/andrem000/iLuvRags.git"
!git clone -q $REPO_URL
%cd iLuvRags
!pip install -q -r requirements.txt


In [ ]:
# Sanity check: index artifacts present?
import os, json

required = [
    "index/faiss.index",
    "index/embeddings.npy",
    "index/metadata.json",
    "index/texts.json",
]
missing = [p for p in required if not os.path.exists(p)]
print("Missing:" if missing else "All good.", missing)


In [ ]:
# Batch queries demo (pretty print + timings)
!python scripts/main.py --queries data/queries.json --device cuda --pretty --llm Qwen/Qwen2.5-3B-Instruct


In [ ]:
# Evaluation (only if labels file exists) with chart
import os
labels_csv = "data/labels.csv"
if os.path.exists(labels_csv):
    !python scripts/eval_retrieval.py --queries_json data/queries.json --labels_csv data/labels.csv --k 10 --chart
else:
    print("No labels.csv present; skip eval.")


# Alpha sweeper (choose best fusion α)
Runs the sweeper over a small grid and plots Recall@k vs α for Tier1/Tier2.


In [ ]:
# Run alpha sweeper (requires labels)
import os
if os.path.exists("data/labels.csv"):
    !python scripts/alpha_sweeper.py --queries_json data/queries.json --labels_csv data/labels.csv --k 10 --alphas 0.5,0.65,0.7,0.8 --chart
else:
    print("No labels.csv present; skip alpha sweeper.")
